# 神经网络的反向传播

## 导数与偏导数

导数是描述函数变化率的概念。设$y=f(x)$,则$y$对$x$的导数定义为:

$$f'(x)=\lim_{h \to 0}\frac{f(x+h)-f(x)}{h}$$

其中$h$是$x$的一个小量,当$h$趋于0时,函数$f(x)$在$x$处的变化率即为导数$f'(x)$。

偏导数是多变量函数的导数概念。设$z=f(x,y)$,则$z$对$x$的偏导数定义为:

$$\frac{\partial z}{\partial x}=\lim_{h \to 0}\frac{f(x+h,y)-f(x,y)}{h}$$

类似地,$z$对$y$的偏导数为:

$$\frac{\partial z}{\partial y}=\lim_{h \to 0}\frac{f(x,y+h)-f(x,y)}{h}$$

偏导数描述了函数中一个自变量对函数值的影响,而其它自变量保持不变。

导数和偏导数都用来描述函数的变化率,但导数针对单变量函数,偏导数用于多变量函数。两者公式形式很相似,都是求函数在某点沿某个方向的变化率极限。

In [2]:
from IPython.display import Image
Image(url= "20.png")

In [2]:
Image(url= "16.png")

## 反向传播中的链式求导

反向传播(Backpropagation)是训练神经网络的重要算法,它的计算过程和原理可以用以下公式表达:

我们假设一个简单的多层前馈神经网络,包含输入层x,隐藏层a,输出层y,以及对应的权重矩阵W和偏置b。

那么这个网络的前向传播可以表示为:

隐藏层:
$$ z^{(1)} = W^{(1)} x + b^{(1)} $$ 
$$ a^{(1)} = f(z^{(1)}) $$

输出层:
$$ z^{(2)} = W^{(2)} a^{(1)} + b^{(2)} $$
$$ y = f(z^{(2)}) $$

其中f表示激活函数。

在反向传播中,我们以损失函数L为参考,计算每个变量对L的梯度贡献:

1. 输出层权重的梯度:
$$ \frac{\partial L}{\partial W^{(2)}} = \frac{\partial L}{\partial y} \frac{\partial y}{\partial z^{(2)}} \frac{\partial z^{(2)}}{\partial W^{(2)}} $$

2. 输出层偏置的梯度:  
$$ \frac{\partial L}{\partial b^{(2)}} = \frac{\partial L}{\partial y} \frac{\partial y}{\partial z^{(2)}} \frac{\partial z^{(2)}}{\partial b^{(2)}} $$

3. 隐藏层权重的梯度:
$$ \frac{\partial L}{\partial W^{(1)}} = \frac{\partial L}{\partial z^{(2)}} \frac{\partial z^{(2)}}{\partial a^{(1)}} \frac{\partial a^{(1)}}{\partial z^{(1)}} \frac{\partial z^{(1)}}{\partial W^{(1)}}$$

4. 隐藏层偏置的梯度:
$$ \frac{\partial L}{\partial b^{(1)}} = \frac{\partial L}{\partial z^{(2)}} \frac{\partial z^{(2)}}{\partial a^{(1)}} \frac{\partial a^{(1)}}{\partial z^{(1)}} \frac{\partial z^{(1)}}{\partial b^{(1)}}$$

由此可更新每个参数,使loss函数L最小化。

反向传播自动进行链式法则的梯度计算,实现了有效的深度网络训练。这是它被广泛采用的关键原因。


* loss.backward()
* net.h1.weight.grad 

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

In [3]:
x=torch.tensor(1.,requires_grad=True)# requires_grad=True表示需要计算梯度
y=x**3
torch.autograd.grad(y,x)# 计算y对单个值x的梯度
# 单变量的情况下，梯度即为导数，故y的导数为3x^2，当x=1时，y的导数为3

(tensor(3.),)

In [4]:
class Model0(nn.Module):
    def __init__(self,in_features=10,out_features=2):
        super(Model0,self).__init__()
        self.h1=nn.Linear(3,3,bias=True)
        self.h2=nn.Linear(3,2,bias=True)
        self.out=nn.Linear(2,3,bias=True)

    def forward(self, x):
        h1_out=self.h1(x)
        h1_out_r=torch.relu(h1_out)
        h2_out=self.h2(h1_out_r)
        h2_out_r=torch.relu(h2_out)
        out_out=self.out(h2_out_r)
        #out_out_s=torch.softmax(out_out,dim=1)
        return out_out

In [5]:
x=torch.tensor([[1.0,2.0,1.5],[3.0,4.0,2.3],[5.0,6.0,1.2]])
z=torch.tensor([0,2,1])

In [6]:
net=Model0(3,3)
net(x)
output=net.forward(x)
#print(output)
criterion = nn.CrossEntropyLoss()
loss = criterion(output,z)#在PyTorch中,有些情况下目标向量(target)需要使用.long()方法转换为长整型tensor(long tensor),
loss.backward()
net.h1.weight.grad 


tensor([[-0.0060, -0.0119, -0.0089],
        [-0.0775, -0.0874, -0.0108],
        [ 0.0724,  0.0817,  0.0101]])

In [7]:
Image(url= "20.png")

## 梯度下降算法


### 梯度向量

梯度向量的方向和大小确定方式可以用以下公式解释:

假设有目标函数$f(\mathbf{x})$,其中$\mathbf{x}$是$n$维向量。

则目标函数$f$相对于$\mathbf{x}$的梯度是一个向量:

$$\nabla_{\mathbf{x}} f = \begin{bmatrix} \frac{\partial f}{\partial x_1} \\ \frac{\partial f}{\partial x_2} \\ \vdots \\ \frac{\partial f}{\partial x_n} \end{bmatrix}$$

其中:

- $\frac{\partial f}{\partial x_i}$表示$f$相对于$x_i$的偏导数

- 梯度向量的每个元素表示目标函数相对于对应变量的变化率

梯度向量方向性质:

- 梯度方向与目标函数增长最快的方向相反

- 沿负梯度方向移动可以使目标函数值下降最快

梯度向量大小性质:

- 梯度大小表示目标函数沿该方向的变化率

- 梯度越大,目标函数越“陡峭”

所以梯度向量充分表示了目标函数的局部变化,方向和大小的合理利用都有利于设计优化算法。



### 对于权重参数的梯度下降的更新公式

假设一个优化问题,其目标是最小化损失函数$L(\omega)$,其中ω表示模型的参数。

则梯度下降法的迭代更新公式为:

$$\omega^{(t+1)} = \omega^{(t)} - \eta \nabla_{\omega} L(\omega^{(t)})$$

其中:

- $\omega^{(t)}$: 模型参数在第$t$次迭代的值  
- $\eta$: 学习率,确定更新步长
- $\nabla_{\omega} L(\omega^{(t)})$: $L$相对于$\omega$的梯度向量
- $t$: 迭代次数

梯度下降的迭代过程是:

1. 初始化参数$\omega$ 

2. 计算损失函数$L(\omega)$关于参数$\omega$的梯度$\nabla_{\omega} L(\omega)$

3. 沿负梯度方向更新参数,步长为学习率$\eta$

4. 重复2-3,直到损失函数收敛或达到预定迭代次数

* net.h1.weight.grad
* net.h1.weight.data
* lr  dw  w  gamma
* v = torch.zeros(dw.shape[0],dw.shape[1])

In [27]:
dw=net.h1.weight.grad
lr=0.1
w0=net.h1.weight.data
print(w0)
w1=w0-lr*dw
w1

tensor([[-0.3902,  0.3928,  0.2833],
        [-0.2951,  0.2564,  0.2619],
        [ 0.0678,  0.1884, -0.0510]])


tensor([[-0.4063,  0.3714,  0.2793],
        [-0.2671,  0.2932,  0.2683],
        [ 0.0271,  0.1356, -0.0597]])

In [28]:
Image(url= "21.png")

### 动量法梯度下降

基于动量法(Momentum)的梯度下降可以使用动量项加速学习,提高收敛速度。其公式表达如下:

假设参数为$\omega$,损失函数为$J(\omega)$,学习率为$\eta$。

基础梯度下降更新:

$$\omega_{t+1} = \omega_{t} - \eta \nabla_\omega J(\omega_{t}) $$

引入动量项$\nu$,动量参数为$\gamma$:

$$\nu_{t+1} = \gamma \nu_{t} + \eta \nabla_\omega J(\omega_{t})$$

$$\omega_{t+1} = \omega_{t} - \nu_{t+1}$$

动量法的思想:

- 使用$\nu$存储历史梯度作为动量
- $\nu$平滑更新,抑制梯度震荡
- $\gamma$控制历史梯度的使用比例

优点:

- 加速学习,提高收敛速度
- 有效抑制共线性等问题造成的震荡

动量法利用历史梯度产生动能,帮助优化器逃离局部最优,是改进梯度下降的常用技巧。

* net.h1.weight.grad
* net.h1.weight.data
* lr=$\eta$  dw  w  gamma=$\gamma$
* v = torch.zeros(dw.shape[0],dw.shape[1])

In [29]:
lr=0.1
gamma=0.9
v=torch.zeros(dw.shape[0],dw.shape[1])
dw=net.h1.weight.grad
w0=net.h1.weight.data
v1=v*gamma+lr*dw
w1=w0-v1
w1

tensor([[-0.4063,  0.3714,  0.2793],
        [-0.2671,  0.2932,  0.2683],
        [ 0.0271,  0.1356, -0.0597]])

### torch.optim

https://zhuanlan.zhihu.com/p/346205754

* 基本用法

优化器主要是在模型训练阶段对模型可学习参数进行更新, 常用优化器有 SGD，RMSprop，Adam等

优化器初始化时传入传入模型的可学习参数，以及其他超参数如 lr，momentum等

在训练过程中先调用 optimizer.zero_grad() 清空梯度，再调用 loss.backward() 反向传播，最后调用 optimizer.step()更新模型参数

* 父类Optimizer 基本原理

Optimizer 是所有优化器的父类，它主要有如下公共方法:

add_param_group(param_group): 添加模型可学习参数组

step(closure): 进行一次参数更新

zero_grad(): 清空上次迭代记录的梯度信息

state_dict(): 返回 dict 结构的参数状态

load_state_dict(state_dict): 加载 dict 结构的参数状态

* optim.SGD(net.parameters() , lr=lr , momentum = gamma)
* opt.zero_grad()
* loss.backward()
* opt.step()

In [8]:
class Model0(nn.Module):
    def __init__(self,in_features=10,out_features=2):
        super(Model0,self).__init__()
        self.h1=nn.Linear(3,3,bias=True)
        self.h2=nn.Linear(3,2,bias=True)
        self.out=nn.Linear(2,3,bias=True)

    def forward(self, x):
        h1_out=self.h1(x)
        h1_out_r=torch.relu(h1_out)
        h2_out=self.h2(h1_out_r)
        h2_out_r=torch.relu(h2_out)
        out_out=self.out(h2_out_r)
        #out_out_s=torch.softmax(out_out,dim=1)
        return out_out

In [9]:
x=torch.tensor([[1.0,2.0,1.5],[3.0,4.0,2.3],[5.0,6.0,1.2]])
z=torch.tensor([0,2,1])

In [17]:
lr=0.01
gamma=0.9

net=Model0(3,3)
net(x)
#print(output)
criterion = nn.CrossEntropyLoss()
opt=optim.SGD(net.parameters() , lr=lr , momentum = gamma)# 表示优化器将优化net的所有参数

for i in range(100):
    output=net.forward(x)
    loss = criterion(output,z)# 计算损失
    # 在PyTorch中,有些情况下目标向量(target)需要使用.long()方法转换为长整型tensor(long tensor),
    #print(net.h1.weight.data)
    opt.zero_grad()# 清除上一次迭代产生的梯度信息
    loss.backward()# 反向传播,计算梯度
    opt.step()# 更新模型参数--各层的w
    print(loss)
    #print(net.h1.weight.data)

tensor(1.2060, grad_fn=<NllLossBackward0>)
tensor(1.2035, grad_fn=<NllLossBackward0>)
tensor(1.1987, grad_fn=<NllLossBackward0>)
tensor(1.1919, grad_fn=<NllLossBackward0>)
tensor(1.1832, grad_fn=<NllLossBackward0>)
tensor(1.1728, grad_fn=<NllLossBackward0>)
tensor(1.1608, grad_fn=<NllLossBackward0>)
tensor(1.1474, grad_fn=<NllLossBackward0>)
tensor(1.1328, grad_fn=<NllLossBackward0>)
tensor(1.1170, grad_fn=<NllLossBackward0>)
tensor(1.1004, grad_fn=<NllLossBackward0>)
tensor(1.0832, grad_fn=<NllLossBackward0>)
tensor(1.0659, grad_fn=<NllLossBackward0>)
tensor(1.0489, grad_fn=<NllLossBackward0>)
tensor(1.0328, grad_fn=<NllLossBackward0>)
tensor(1.0182, grad_fn=<NllLossBackward0>)
tensor(1.0059, grad_fn=<NllLossBackward0>)
tensor(0.9963, grad_fn=<NllLossBackward0>)
tensor(0.9897, grad_fn=<NllLossBackward0>)
tensor(0.9859, grad_fn=<NllLossBackward0>)
tensor(0.9841, grad_fn=<NllLossBackward0>)
tensor(0.9832, grad_fn=<NllLossBackward0>)
tensor(0.9822, grad_fn=<NllLossBackward0>)
tensor(0.97